In [8]:
import numpy

In [9]:
class Network:
    pass

In [10]:
class Layer_Dense:
    pass

In [11]:
class ActivationFunction:
    pass

In [12]:
class ReLU:
    pass

In [13]:
class Softmax:
    pass

In [14]:
class Sigmoid:
    pass

In [ ]:
class Loss:
    pass

In [ ]:
class MeanSquaredError:
    pass

In [ ]:
class CategorialCrossEntropy:
    pass